## Forced particle on a circle

Pierre de Buyl  
Instituut voor Theoretische Fysica, KU Leuven

The code and notebooks in the repository `2018_nonequilibrium_simulations`
constitute supplementary material for the lecture notes
*Langevin simulations for nonequilibrium physics*.
See the [README.md](README.md) file for more information.
See the lecture notes (link posted soon, visit [my website](http://pdebuyl.be/)
for updates.

This notebook contains the example of a colloidal particle in a toroidal trap.

In [ ]:
# import libraries and set default figure parameters
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import algorithms
import math

plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.subplot.top'] = 0.95
plt.rcParams['figure.subplot.right'] = 0.95

plt.rcParams['font.size'] = 16

In [ ]:
%load_ext cython

In [ ]:

def U(theta, A):
    return A*np.sin(theta)

def f(theta, A):
    return -A*np.cos(theta)

def fixed_A_force(A):
    def force(theta):
        return f(theta, A)
    return force

def euler(x, f, mu, T, dt, n):
    x = np.array(x).copy()
    shape = x.shape
    step = np.sqrt(2*mu*T*dt)
    for i in range(n):
        g0 = np.dot(step, np.random.normal(size=shape))
        f1 = f(x)
        x = x + dt*np.dot(mu, f1) + g0
    return x


In [ ]:
%%cython

cimport algorithms
from libc.math cimport cos

cdef class cy_A_force(algorithms.cyfunc_nd):
    cdef double A
    def __init__(self, A):
        self.A = A
    cpdef void force(self, double[::1] x, double[::1] f):
        cdef int i
        for i in range(x.shape[0]):
            f[i] = -self.A*cos(x[i])

cdef class cy_A_drive(algorithms.cyfunc_nd):
    cdef double A
    cdef double F
    def __init__(self, A, F):
        self.A = A
        self.F = F
    cpdef void force(self, double[::1] x, double[::1] f):
        cdef int i
        for i in range(x.shape[0]):
            f[i] = -self.A*cos(x[i]) + self.F


In [ ]:
mu = np.array([1], dtype=float)
T = 1.0
dt = 1e-2
nsteps = 50
npoints = 100000

theta = np.array([0], dtype=float)

theta = algorithms.integrate_euler(theta, mu, T, dt, npoints, nsteps, cy_A_force(1.0))


In [ ]:
plt.plot(theta)

In [ ]:
periodic_theta = np.mod(theta, 2*np.pi)
plt.hist(periodic_theta, density=True, bins=32, histtype='step');

thr = np.linspace(0, 2*np.pi, 100)
p = np.exp(-U(thr, 1))
p /= np.sum(p)*(thr[1]-thr[0])
plt.plot(thr, p)


In [ ]:
periodic_theta = np.mod